In [ ]:
!pip install category_encoders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder 
from datetime import datetime
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import time
from sklearn.preprocessing import LabelEncoder
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def mergedata(file1,file2,file3):
  data3 = pd.read_csv(file1)
  data1 = pd.read_csv(file2) #name
  data2 = pd.read_csv(file3) #movie
  data1.rename(columns={'name': 'movie_title'}, inplace=True)
  data2.rename(columns={'movie': 'movie_title'}, inplace=True)
  data = pd.merge(data3,data1[['movie_title','director']],on='movie_title', how='left')
  data_actors= data2.groupby('movie_title')['voice-actor'].agg(list)
  my_data = pd.merge(data,data_actors,on='movie_title', how='left')
  my_data['release_date'] = pd.to_datetime(my_data['release_date'])
  my_data['release_date'] = pd.DatetimeIndex(my_data['release_date']).year
  for index, row in my_data.iterrows():
    if(row["release_date"] > 2022):
      my_data.at[index,"release_date"] = row["release_date"] - 100
  return my_data


In [ ]:
my_data = mergedata("movies-tas-regression-test.csv","movie-director.csv","movie-voice-actors.csv")

In [ ]:
import pickle
actorFq = open("actorFq.pkl", "rb")
actorfq = pickle.load(actorFq)
actorFq.close()

directorFq = open("directorFq.pkl", "rb")
directorfq = pickle.load(directorFq)
directorFq.close()


In [ ]:

newDirector = dict()
def directorEncoding():
  for index, row in my_data.iterrows():    
    if row["director"] in directorfq.keys():
      continue
    else:
        newDirector[row["director"]]=0

  for idx,row in my_data.iterrows():
    if pd.isna(my_data.iloc[idx,5]):
      continue
    elif my_data.iloc[idx,5] in directorfq.keys():
      my_data.at[idx,"director"]=directorfq[ my_data.at[idx,"director"]]
    else:
      my_data.at[idx,"director"]=newDirector[ my_data.at[idx,"director"]]
directorEncoding()

In [ ]:
ratingCols = open("ratingCols.pkl", "rb")
ratingcols = pickle.load(ratingCols)
ratingCols.close()

otherCols = open("otherRatingCols.pkl", "rb")
otherRatingcols = pickle.load(otherCols)
otherCols.close()


In [ ]:
genreCols = open("genreCols.pkl", "rb")
genrecols = pickle.load(genreCols)
genreCols.close()

otherGenreCols = open("otherGenreCols.pkl", "rb")
otherGenrecols = pickle.load(otherGenreCols)
otherGenreCols.close()


In [ ]:
def ratingEncoding():
  for l in ratingcols:
    my_data[l] = np.where(my_data['MPAA_rating']==l,1,0)

  my_data["other_rating"] = " "
  for idx,row in my_data.iterrows():
    if pd.isna(my_data.iloc[idx,3]):
      my_data["other_rating"][idx]=0
    if my_data["MPAA_rating"][idx] in otherRatingcols:
      my_data["other_rating"][idx]=1
    else:
      my_data["other_rating"][idx]=0
ratingEncoding()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def genreEncoding():
  for l in genrecols:
    my_data[l] = np.where(my_data['genre']==l,1,0)

  my_data["other_genre"] = " "
  for idx,row in my_data.iterrows():
    if pd.isna(my_data.iloc[idx,2]):
      my_data["other_genre"][idx]=0
    if my_data["genre"][idx] in otherGenrecols:
      my_data["other_genre"][idx]=1
    else:
      my_data["other_genre"][idx]=0
genreEncoding()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
my_data= my_data.drop("genre",axis = 1)
my_data= my_data.drop("MPAA_rating",axis = 1)


In [ ]:
newActors = dict()
def actorEncoding():
  for index, row in my_data.iterrows(): 
    if  type  (row['voice-actor'])is list:
      for actor in row['voice-actor']:
        if actor in actorfq.keys():
          continue
        else:     
            newActors[actor] = 0

  for index, row in my_data.iterrows():
    if  type(row['voice-actor'])is list:
      fq = 0
      for actor in row['voice-actor']:
        if actor in actorfq.keys():
          fq = fq + actorfq[actor]
          my_data.at[index,"voice-actor"] = fq
        else:
          fq = fq + newActors[actor]
          my_data.at[index,"voice-actor"] = fq
actorEncoding()

In [ ]:
my_data= my_data.drop("movie_title",axis = 1)

In [ ]:
my_data

,release_date,revenue,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,1997,"$3,257,546",NaN,NaN,1,0,0,0,0,1,0
1,2014,"$37,607,865",NaN,NaN,1,0,0,0,0,1,0
2,2002,"$109,691,666",NaN,NaN,0,0,1,0,0,1,0
3,1990,"$39,433,357",NaN,NaN,0,0,1,1,0,0,0
4,1993,"$115,781,734",NaN,NaN,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
108,1992,"$9,907,922",NaN,NaN,0,0,1,0,0,0,1
109,1991,"$22,580,472",NaN,NaN,0,0,1,0,0,0,1
110,1999,"$32,228,598",NaN,NaN,0,0,1,0,1,0,0
111,1991,"$14,617,333",NaN,NaN,0,0,0,0,0,1,0


In [ ]:
label = my_data["revenue"]
my_data = my_data.drop("revenue",axis = 1)

In [ ]:
label = label.str.replace('$', '')
label =label.str.replace(',', '').astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
knnimputerFile = open("knnimputer.pkl", "rb")
knnimputer = pickle.load(knnimputerFile)
knnimputerFile.close()


In [ ]:
def dataImputation(my_data):
  Xtrans = knnimputer.transform(my_data)
  my_data = pd.DataFrame(Xtrans, columns=my_data.columns)
  for col in my_data.columns:
    my_data[col] = my_data[col].astype(int)
  return my_data
my_data = dataImputation(my_data)

In [ ]:
x = my_data

In [ ]:
droppedColsFile = open("droppedColsForRegg.pkl", "rb")
droppedcols = pickle.load(droppedColsFile)
droppedColsFile.close()


In [ ]:
droppedcols

['director', 'PG', 'PG-13', 'Comedy', 'Drama']

In [ ]:
for col in droppedcols:
  x = x.drop(col, axis = 1)

In [ ]:
model = open("poly.sav", "rb")
poly = pickle.load(model)
model.close()

model = open("polyFeatures.sav", "rb")
poly_features =  pickle.load(model)
model.close()

ypred = poly.predict(poly_features.transform(x))

print('Mean Square Error test', metrics.mean_squared_error(label, ypred))
print("r2 score for the test",metrics.r2_score(label, ypred))

Mean Square Error test 3.1097894924424616e+16
r2 score for the test 0.4011131304089207


In [ ]:
model = open("LR.sav", "rb")
cls = pickle.load(model)
model.close()
prediction= cls.predict(x)
print('Mean Square Error', metrics.mean_squared_error(label, prediction))
print("r2 score for the test",metrics.r2_score(label, prediction))

Mean Square Error 4.541970562708585e+16
r2 score for the test 0.12530203774694704
